<h1><center>Neighborhoods in City of Toronto</center></h1>

<Bold>__Welcome! This notebook is the final Capstone Project for Applied Data Science Capstone Course__</Bold>

### Segmenting and Clustering Neighbourhoods in Toronto

##### Import and install required libraries

In [2]:
!pip install lxml
import pandas as pd # library for data analsysis
import numpy as np

     |████████████████████████████████| 5.8MB 23.9MB/s eta 0:00:01     |████████████████████████▋       | 4.4MB 23.9MB/s eta 0:00:01


In [3]:
#Import Pandas, Numpy and Requests libraries
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda update -n base -c defaults conda -y
#!conda -y install package

print("Libraries imported")


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

## 1. Download and Explore Dataset

##### Read HTML table to Pandas dataframe

In [5]:
#Read HTML table to pandas dataframe
df_read = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]

##### Remove rows with Borough value as 'Not assigned'

In [6]:
#Remove rows with Borough value as 'Not assigned'
df_city = df_read[df_read.Borough != 'Not assigned']

##### Groupby Postcode and Borough and combine Neighbourhood values

In [7]:
#Group the dataframe by Postcode and Borough and join Neighbourhood value with ',' separated values
df_city = df_city.groupby(['Postcode', 'Borough'], as_index=False, sort=False).agg(','.join)

##### Replace Neighbourhood value with Borough value when Neighbouhood is 'Not assigned'

In [8]:
#Replace 'Neighbourhood' value with 'Borough' value when Neighbourhood='Not assigned'
df_city['Neighbourhood'] = np.where((df_city.Neighbourhood == 'Not assigned'), df_city['Borough'], df_city.Neighbourhood)
df_city

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


##### Print the number of rows and columns in the dataframe

In [9]:
print("Number of rows in the dataframe: ", df_city.shape[0])
print("Number of columns in the dataframe: ", df_city.shape[1])

Number of rows in the dataframe:  103
Number of columns in the dataframe:  3


##### Retrieve Latitude and Longitude

In [10]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
print ('Data read into a pandas dataframe!')
df_geo.rename(columns = {'Postal Code':'Postcode'}, inplace = True)
df_geo

Data read into a pandas dataframe!


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


##### Merge dataframe with city and the dataframe with latitude,longitude

In [11]:
df_comb = pd.merge(df_city, df_geo, how='inner', on = 'Postcode')
df_comb

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
